# Delta Lake Lab 
## Unit 5: Schema Enforcement & Evolution

In the previous unit, we -
1. Did a delete operation on the base delta table and learned what happens to the underlying parquet and reviewed the transaction log
2. Did an insert operation on the base delta table and learned what happens to the underlying parquet and reviewed the transaction log
3. Did an update operation on the base delta table and learned what happens to the underlying parquet and reviewed the transaction log
4. Did a merge (upsert) operation on the base delta table and learned what happens to the underlying parquet and reviewed the transaction log

In this unit, we will -
1. Study schema enforcement
2. And schema evolution possible with delta lake


### 1. Imports

In [ ]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [ ]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

### 3. Declare variables

In [ ]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

In [ ]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

In [ ]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

In [ ]:
ACCOUNT_NAME = "YOUR_ACCOUNT_NAME"

In [ ]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}-{ACCOUNT_NAME}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

### 4. Existing schema

In [ ]:
spark.sql("DESCRIBE FORMATTED "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta").show(truncate=False)

### 5. Attempt to modify the schema

In [ ]:
# Add a column called collateral_value
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta")
schemaEvolvedDF.show(3)

In [ ]:
# Attempt to append to the table
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)

### 6. Supply "mergeSchema" option

In [ ]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

In [ ]:
schemaEvolvedDF.write.option("mergeSchema",True).format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)

In [ ]:
spark.sql("SELECT * FROM "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta").show(truncate=False)

In [ ]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

In [ ]:
# Lets look at our datalake and changes from the above execution
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

There is one extra parquet, containing the data with the new column, and one new transaction log


Lets look at the log-

In [ ]:
!gsutil cat {DATA_LAKE_ROOT_PATH}/delta-consumable/_delta_log/00000000000000000006.json

In [ ]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log


In [ ]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

In [ ]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)


In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

In [ ]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part*

In [ ]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

In [ ]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

In [ ]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

In [ ]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

In [ ]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

In [ ]:
# Lets add data again till we hit the 10th version of the table
schemaEvolvedDF = sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as collateral_value from "+ ACCOUNT_NAME +"_loan_db.loans_by_state_delta")
schemaEvolvedDF.write.format("delta").mode("append").save(DELTA_LAKE_DIR_ROOT)
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log

In [ ]:
# Data file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

In [ ]:
# Delta log count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

Note how there is a parquet file created - this is a compacted version of table changes 0-9
Delta will load this parquet into memory, to avoid having to read too many small files
At any given point of time, delta will not read more than 10 json transaction logs, but will read all the parquet transaction logs.

### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK